<a href="https://colab.research.google.com/github/pannettonne/Portfolio/blob/main/LaSalle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import importlib.util
import subprocess
#import pkg_resources
import sys
print("Empiezo")

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import os
import yagmail
from premailer import Premailer

#import pyjuice as juice
import css_inline

import re
import unicodedata
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

def minify_html(html: str) -> str:
    html = re.sub(r'>\s+<', '><', html)  # elimina espacios/saltos entre etiquetas
    html = re.sub(r'\s{2,}', ' ', html)  # colapsa espacios repetidos
    return html.strip()

def norm_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s).replace("\xa0", " ")
    s = s.replace("–", "-").replace("—", "-").replace("\u2011","-").replace("\u200b","")
    return " ".join(s.split())

os.environ['KEYRING_BACKEND'] = 'keyrings.alt.chainer.ChainerBackend'

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, JavascriptException
import time

from webdriver_manager.chrome import ChromeDriverManager

# --- WebDriver ---
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = '/usr/bin/chromium-browser'
# logs de navegador (por si quieres ver errores JS)
chrome_options.set_capability('goog:loggingPrefs', {'browser': 'ALL'})

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=chrome_options)

# ====== METEO (Open-Meteo) ======
import json
from datetime import datetime
def _http_get(url):
    try:
        import requests
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        return r.text
    except Exception:
        # fallback sin requests
        import urllib.request
        with urllib.request.urlopen(url, timeout=15) as resp:
            return resp.read().decode("utf-8")

def _wcode_text_emoji(code: int) -> str:
    # Mapa abreviado WMO → texto + emoji
    # https://open-meteo.com/en/docs#api_form
    m = {
        0: ("Despejado", "☀️"),
        1: ("Más bien despejado", "🌤️"),
        2: ("Parcialmente nublado", "⛅"),
        3: ("Nublado", "☁️"),
        45: ("Niebla", "🌫️"), 48: ("Niebla con escarcha", "🌫️"),
        51: ("Llovizna débil", "🌦️"), 53: ("Llovizna", "🌦️"), 55: ("Llovizna intensa", "🌦️"),
        56: ("Llovizna helada", "🌧️"), 57: ("Llovizna helada", "🌧️"),
        61: ("Lluvia débil", "🌦️"), 63: ("Lluvia", "🌧️"), 65: ("Lluvia intensa", "🌧️"),
        66: ("Lluvia helada", "🌨️"), 67: ("Lluvia helada", "🌨️"),
        71: ("Nieve débil", "🌨️"), 73: ("Nieve", "🌨️"), 75: ("Nieve intensa", "🌨️"),
        77: ("Granos de nieve", "🌨️"),
        80: ("Chubascos débiles", "🌧️"), 81: ("Chubascos", "🌧️"), 82: ("Chubascos intensos", "🌧️"),
        85: ("Chubascos de nieve", "🌨️"), 86: ("Chubascos de nieve intensos", "🌨️"),
        95: ("Tormenta", "⛈️"), 96: ("Tormenta con granizo", "⛈️"), 99: ("Tormenta con granizo fuerte", "⛈️"),
    }
    return " ".join(m.get(code, ("", "❓")))

def meteo_moncloa_semana_html() -> str:
    print("METEO")
    """
    Devuelve una tabla HTML con la previsión diaria (7-10 días) para Moncloa (Madrid).
    Fuente: Open-Meteo (sin API key).
    """
    # Moncloa-Aravaca aprox:
    lat, lon = 40.437, -3.732
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        "&daily=weathercode,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max"
        "&timezone=Europe%2FMadrid"
    )
    try:
        txt = _http_get(url)
        data = json.loads(txt)
        d = data["daily"]
        fechas = d["time"]
        wcode = d["weathercode"]
        tmax = d["temperature_2m_max"]
        tmin = d["temperature_2m_min"]
        prcp = d["precipitation_sum"]
        wmax = d["windspeed_10m_max"]

        # Construir filas
        filas = []
        for i in range(len(fechas)):
            dt = datetime.strptime(fechas[i], "%Y-%m-%d")
            dia_nombre = ["Lun","Mar","Mié","Jue","Vie","Sáb","Dom"][dt.weekday()]
            fecha_fmt = dt.strftime("%d/%m")
            wtxt = _wcode_text_emoji(int(wcode[i]))
            filas.append(
                f"<tr>"
                f"<td>{dia_nombre} {fecha_fmt}</td>"
                f"<td>{wtxt}</td>"
                f"<td style='text-align:right;'>{round(tmax[i])}°</td>"
                f"<td style='text-align:right;'> {round(tmin[i])}°</td>"
                f"<td style='text-align:right;'>{prcp[i]:.1f} mm</td>"
                f"<td style='text-align:right;'>{round(wmax[i])} km/h</td>"
                f"</tr>"
            )

        tabla = (
            "<table style='width:100%; border-collapse:collapse;' class='table table-hover table-bordered'>"
            "<thead>"
            "<tr>"
            "<th>Día</th><th>Estado</th><th>Máx</th><th>Mín</th><th>Precip</th><th>Viento máx</th>"
            "</tr>"
            "</thead>"
            "<tbody>"
            + "".join(filas) +
            "</tbody></table>"
        )
        # Encabezado
        bloque = (
            "<h1 style='font-weight:bold; margin-top:16px;'>Tiempo en Moncloa (Madrid) · Semana</h1>"
            + tabla
        )
        return bloque
    except Exception as e:
        return f"<p><em>No se pudo cargar el tiempo (Open-Meteo): {e}</em></p>"
# ====== FIN METEO ======

def dump_debug(name="debug"):
    try:
        path_png = f"/tmp/{name}.png"
        path_html = f"/tmp/{name}.html"
        driver.save_screenshot(path_png)
        with open(path_html, "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print(f"[DEBUG] Screenshot: {path_png}")
        print(f"[DEBUG] HTML: {path_html}")
        sys.stdout.flush()
    except Exception as e:
        print(f"[DEBUG] No se pudo guardar debug: {e}", flush=True)

# -------- Helpers UI --------
def esperar_tabla(timeout=20):
    print("[DBG] Esperando tabla en #div_padres…")
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//*[@id="div_padres"]//table[contains(@class,"table") and contains(@class,"table-bordered")]'
        ))
    )
    print("[DBG] Tabla detectada.")

def click_vista_diaria(timeout=10):
    print("[DBG] Click 'Vista diaria'…")
    btn = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(normalize-space(.), "Vista diaria")]'))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    driver.execute_script("arguments[0].click();", btn)
    esperar_tabla()
    print("[DBG] Click 'Vista diaria' OK.")

def click_vista_semanal(timeout=10):
    print("[DBG] Click 'Vista semanal'…")
    btn = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(normalize-space(.), "Vista semanal")]'))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    driver.execute_script("arguments[0].click();", btn)
    esperar_tabla()
    print("[DBG] Click 'Vista semanal' OK.")

def set_fecha_en_input_y_disparar_change(fecha_str: str):
    """
    Pone fecha en #fecha_incidencia y dispara el onchange para que recargue la vista diaria.
    Usa jQuery si está presente; si no, Event('change') nativo.
    """
    print(f"[DBG] Set #fecha_incidencia = {fecha_str} + change()…")
    js = """
        const f = arguments[0];
        const inp = document.getElementById('fecha_incidencia');
        if (!inp) return 'NO_INPUT';
        inp.value = f;
        if (window.jQuery) {
            jQuery(inp).trigger('change');
            return 'JQ_CHANGE';
        } else {
            const ev = new Event('change', {bubbles:true});
            inp.dispatchEvent(ev);
            return 'NATIVE_CHANGE';
        }
    """
    res = driver.execute_script(js, fecha_str)
    print(f"[DBG] Resultado set/trigger: {res}")

def esperar_header_de_fecha(fecha_str: str, timeout=25):
    print(f"[DBG] Esperando <th> con '{fecha_str}'…")
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((
            By.XPATH, f'//*[@id="div_padres"]//th[contains(normalize-space(.), "{fecha_str}")]'
        ))
    )
    print("[DBG] Header con la fecha detectado.")

def get_tabla_html() -> str:
    tabla_elem = driver.find_element(
        By.XPATH,
        '//*[@id="div_padres"]//table[contains(@class,"table") and contains(@class,"table-bordered")]'
    )
    html = tabla_elem.get_attribute("outerHTML")
    print(f"[DBG] Longitud HTML tabla: {len(html)}")
    return html

def quitar_notas_personales(html: str) -> str:
    """
    Elimina filas <tr> cuyo PRIMER <th> (no el header de día) sea 'Notas personales'
    (insensible a mayúsculas y espacios).
    """
    try:
        soup = BeautifulSoup(html, "html.parser")
        for tr in soup.select("tr"):
            # primer <th> de la fila (si existe)
            first_th = tr.find("th")
            if not first_th:
                continue
            label = norm_text(first_th.get_text(" ", strip=True)).lower()
            # match exacto al rótulo de fila, no al header de día
            if label == "notas personales":
                tr.decompose()
        return str(soup)
    except Exception:
        return html


def cargar_vista_semanal_para(fecha_str: str) -> str:
    """
    Entra en vista semanal (botón) y devuelve la tabla semanal.
    Intenta además forzar por JS a la semana de esa fecha (opcional).
    """
    click_vista_semanal()
    try:
        print(f"[DBG] (opcional) Forzando semanal JS para {fecha_str}…")
        driver.execute_script(
            'return CargaDiv("div_padres","padre","padre_trabajo","fecha", arguments[0], "tipo", 1);',
            fecha_str
        )
        # no todas las semanales muestran la fecha exacta en un <th>; no bloqueamos si no aparece
        time.sleep(1.0)
    except Exception as e:
        print(f"[DBG] No forcé semanal por JS: {e}")
    return get_tabla_html()

def cargar_vista_diaria_para(fecha_str: str) -> str:
    print("Cargar vista diaria")
    print(fecha_str)
    """
    Entra en vista diaria, fija #fecha_incidencia a fecha_str, dispara change,
    espera el encabezado de esa fecha y devuelve la tabla HTML del día.
    """
    click_vista_diaria()
    # asegurar input visible
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'fecha_incidencia')))
    set_fecha_en_input_y_disparar_change(fecha_str)
    esperar_header_de_fecha(fecha_str, timeout=25)
    return get_tabla_html()

try:
    # --- Login ---
    driver.get('https://lasallesanrafael.sallenet.org')
    time.sleep(3)
    username_field = driver.find_element(By.ID, 'username')
    password_field = driver.find_element(By.ID, 'password')

    # *** Manteniendo como lo tenías ***
    username_field.send_keys('davidpaniaguadominguez')
    password_field.send_keys('Valhalla99!')

    driver.find_element(By.ID, 'loginbtn').click()
    time.sleep(10)
    print("Login exitoso")

    # --- Padres -> Trabajo ---
    driver.get('https://lasallesanrafael.sallenet.org/mod/sallenet/modulos/padres/')
    time.sleep(10)
    print("Navegación a 'padres' OK")

    enlace_trabajo = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Trabajo')]"))
    )
    driver.execute_script("arguments[0].click();", enlace_trabajo)
    esperar_tabla()

    # --- Fecha hoy (Europe/Madrid) ---
    try:
        from zoneinfo import ZoneInfo
        hoy_str = datetime.now(ZoneInfo("Europe/Madrid")).strftime("%d/%m/%Y")
    except Exception:
        hoy_str = datetime.utcnow().strftime("%d/%m/%Y")
    print(f"[DBG] Hoy: {hoy_str}")

    # 1) Capturar SEMANA primero
    print("[DBG] Cargando VISTA SEMANAL…")
    tabla_semana_html = cargar_vista_semanal_para(hoy_str)
    print("[DBG] Semanal capturada ✅")

    # 2) Cambiar a DIARIA y capturar
    print("[DBG] Cargando VISTA DIARIA…")
    tabla_dia_html = cargar_vista_diaria_para(hoy_str)
    print("[DBG] Diaria capturada ✅")


    # --- Email ---
    estilos_css = """
    <style>
        table { width: 100%; border-collapse: collapse; }
        th, td { border: 1px solid #dddddd; text-align: left; padding: 8px; vertical-align: top; }
        th { background-color: #f2f2f2; }
        tr:nth-child(even) { background-color: #f9f9f9; }
        h1 { margin: 0 0 10px; }
    </style>
    """

    bloque_meteo = meteo_moncloa_semana_html()

    tabla_semana_html = quitar_notas_personales(tabla_semana_html)
    tabla_dia_html    = quitar_notas_personales(tabla_dia_html)

    # ===== NUEVO: transformar tablas capturadas a tu formato (como meteo) =====

    import re
    from bs4 import BeautifulSoup

    # Detecta fechas tipo 01/10/2025 o 01/10/25 o 01/10
    _RE_FECHA = re.compile(r"\b([0-3]?\d/[01]?\d(?:/\d{2,4})?)\b")

    def _es_fila_header_de_dia(tr) -> str | None:
        """
        Header de día = fila con un único <th> que contiene una fecha.
        Devuelve la fecha encontrada o None.
        """
        ths = tr.find_all("th", recursive=False)
        tds = tr.find_all("td", recursive=False)
        if len(ths) == 1 and len(tds) == 0:
            txt = norm_text(ths[0].get_text(" ", strip=True))
            m = _RE_FECHA.search(txt)
            if m:
                return m.group(1)
        return None

    def _celda_html(td) -> str:
        """Devuelve el HTML interno preservando enlaces/listas."""
        try:
            return td.decode_contents().strip()
        except Exception:
            return norm_text(td.get_text(" ", strip=True))

    def _limpia_rotulo(rot: str) -> str:
        rot = norm_text(rot)
        rot = re.sub(r"(?i)^asignatura:\s*", "", rot)
        return rot

    def parsear_bloques_por_dia(html_tabla: str):
    """
    Devuelve: [{'fecha': 'dd/mm[/yyyy]', 'filas': [{'asignatura','tareas','plan'}, ...]}, ...]
    Soporta estructura:
      - <tr><th>ASIGNATURA</th>[<td>...tareas...</td>[<td>...plan...</td>]]</tr>
      - <tr><th>Tareas</th><td>…</td></tr>  (para la asignatura actual)
      - <tr><th>Planificación</th><td>…</td></tr>  (para la asignatura actual)
      - Cabecera de día: <tr><th>29/09/2025</th></tr>
    """
    soup = BeautifulSoup(html_tabla, "html.parser")
    tabla = soup.find("table")
    if not tabla:
        return []

    bloques = []
    actual_bloque = None  # {'fecha': str, 'filas': [...]}
    asignatura_actual = None  # dict de la fila en construcción

    def _push_asignatura():
        nonlocal asignatura_actual
        if asignatura_actual and (asignatura_actual.get('tareas') or asignatura_actual.get('plan') or asignatura_actual.get('asignatura')):
            # Limpia None
            asignatura_actual['tareas'] = asignatura_actual.get('tareas', '')
            asignatura_actual['plan']   = asignatura_actual.get('plan', '')
            actual_bloque['filas'].append(asignatura_actual)
        asignatura_actual = None

    for tr in tabla.find_all("tr", recursive=True):
        # --- 1) ¿Header de día? ---
        fecha_dia = _es_fila_header_de_dia(tr)
        if fecha_dia:
            if actual_bloque:
                _push_asignatura()
            actual_bloque = {'fecha': fecha_dia, 'filas': []}
            bloques.append(actual_bloque)
            asignatura_actual = None
            continue

        ths = tr.find_all("th", recursive=False)
        tds = tr.find_all("td", recursive=False)

        # Ignora filas vacías totales
        if not ths and not tds:
            continue

        # Si aún no hay bloque de día y llega contenido, crea uno genérico
        if not actual_bloque:
            actual_bloque = {'fecha': 'Día', 'filas': []}
            bloques.append(actual_bloque)

        # --- 2) Con TH (rótulo) ---
        if ths:
            rotulo = _limpia_rotulo(norm_text(ths[0].get_text(" ", strip=True))).lower()

            # ¿Fila "Tareas" / "Planificación"?
            if rotulo in {"tareas", "planificación", "planificacion"}:
                # Si no hay asignatura activa, creamos "General"
                if not asignatura_actual:
                    asignatura_actual = {'asignatura': 'General', 'tareas': '', 'plan': ''}

                contenido = _celda_html(tds[0]) if len(tds) >= 1 else ""
                if rotulo == "tareas":
                    # concatena si ya había contenido
                    if asignatura_actual.get('tareas'):
                        asignatura_actual['tareas'] += "<br>" + contenido
                    else:
                        asignatura_actual['tareas'] = contenido
                else:
                    if asignatura_actual.get('plan'):
                        asignatura_actual['plan'] += "<br>" + contenido
                    else:
                        asignatura_actual['plan'] = contenido
                continue

            # Si NO es "Tareas/Planificación", es una **ASIGNATURA**
            # Cerramos la anterior (si la hubiera) y abrimos una nueva
            _push_asignatura()
            nombre_asig = _limpia_rotulo(norm_text(ths[0].get_text(" ", strip=True)))
            asignatura_actual = {'asignatura': nombre_asig, 'tareas': '', 'plan': ''}

            # Si en esa misma fila ya vienen celdas:
            if tds:
                if len(tds) == 1:
                    asignatura_actual['tareas'] = _celda_html(tds[0])
                else:
                    asignatura_actual['tareas'] = _celda_html(tds[0])
                    asignatura_actual['plan']   = _celda_html(tds[1])
            continue

        # --- 3) Filas sin TH (raro, pero por si hay continuaciones) ---
        if tds and asignatura_actual:
            # Si hay una sola celda sin rótulo, la añadimos a 'tareas' por defecto
            cont = _celda_html(tds[0])
            if asignatura_actual.get('tareas'):
                asignatura_actual['tareas'] += "<br>" + cont
            else:
                asignatura_actual['tareas'] = cont

    # cierra la última asignatura del último bloque
    if actual_bloque:
        _push_asignatura()

    # filtra días sin filas
    return [b for b in bloques if b['filas']]


    def _thead_html():
        return (
            "<thead>"
            "<tr>"
            "<th>Asignatura</th>"
            "<th>Tareas</th>"
            "<th>Planificación</th>"
            "</tr>"
            "</thead>"
        )

    def render_tabla_por_dia(fecha: str, filas: list[dict]) -> str:
        """
        Renderiza <h2>Día …</h2> + tabla con clase y estilo como en meteo.
        """
        filas_html = []
        for f in filas:
            filas_html.append(
                "<tr>"
                f"<th>{f['asignatura']}</th>"
                f"<td>{f['tareas'] or ''}</td>"
                f"<td>{f['plan'] or ''}</td>"
                "</tr>"
            )
        tabla = (
            "<table style='width:100%; border-collapse:collapse;' class='table table-hover table-bordered'>"
            + _thead_html() +
            "<tbody>" + "".join(filas_html) + "</tbody>"
            "</table>"
        )
        return f"<h2 style='font-weight:bold; margin:16px 0 8px;'>Día {fecha}</h2>{tabla}"

    def render_bloques(bloques: list[dict]) -> str:
        return "".join(render_tabla_por_dia(b['fecha'], b['filas']) for b in bloques)

    # --- Aplicar al DÍA ACTUAL y a la SEMANA ---
    bloques_dia    = parsear_bloques_por_dia(tabla_dia_html)
    bloques_semana = parsear_bloques_por_dia(tabla_semana_html)

    tabla_html_modificada  = render_bloques(bloques_dia)    if bloques_dia    else "<p><em>Sin datos para el día.</em></p>"
    tabla_html_modificada2 = render_bloques(bloques_semana) if bloques_semana else "<p><em>Sin datos para la semana.</em></p>"
    # ===== FIN NUEVO =====


    #tabla_html_modificada = tabla_dia_html        # DÍA ACTUAL
    #tabla_html_modificada2 = tabla_semana_html    # SEMANA COMPLETA



    yag = yagmail.SMTP('elpaniaguado@gmail.com')

    # *** Manteniendo tus listas ***
    destinatarios = [
        'elpaniaguado@gmail.com',
        'cquinteromoreno@gmail.com',
        'sharonrdc@yahoo.es',
        'abdjimenez@hotmail.com',
        'paradinasmarcos@gmail.com',
        'paulaherediaalcantara@gmail.com',
        'mariofvilladangos@hotmail.com',
        'adrian.alvarez.munoz@gmail.com',
        'nu.abril@gmail.com',
        'lorenitavelayos@gmail.com',
        'judiith22@gmail.com',
        'aransay13@gmail.com',
        'emmaedani@gmail.com',
        'anaisabelmas@hotmail.com',
        'juandiazlorenzo@hotmail.com',
        "lorena.torresc85@gmail.com"
    ]
    destinatarios = ['elpaniaguado@gmail.com']  # ajusta cuál usas

    contenido_html = f"""
    <html>
    <head>{estilos_css}</head>
    <body>
        <h1 style="font-weight:bold;">DIA ACTUAL</h1>
        {tabla_html_modificada}
        <h1 style="font-weight:bold;">SEMANA COMPLETA</h1>
        {tabla_html_modificada2}
        {bloque_meteo}
    </body>
    </html>
    """

    contenido_html_inline = css_inline.inline(contenido_html)
    contenido_html_min = minify_html(contenido_html_inline)

    dia_actual_str = datetime.now().strftime("%d-%m-%Y")
    subject = "Calendario de tareas La Salle 1B " + dia_actual_str

    try:
        yag.send(
            to=destinatarios,  # o destinatarios3
            subject=subject,
            contents=[yagmail.raw(contenido_html_min)]
        )
        print("Correo enviado con éxito a todos los destinatarios.")
    except Exception as e:
        print(f"Error al enviar el correo: {e}")

except Exception as e:
    print("Fallo:", e)
    dump_debug("fatal")
    # logs de navegador por si hay error JS
    try:
        for entry in driver.get_log('browser'):
            print(f"[BROWSER] {entry.get('level')}: {entry.get('message')}")
    except Exception:
        pass
    raise
finally:
    driver.quit()
    print("Fin.")




Empiezo


ModuleNotFoundError: No module named 'yagmail'